In [40]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.core import Dense
import copy

import re
import nltk
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('popular')
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to C:\Users\Sai
[nltk_data]    |     Pavan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to C:\Users\Sai
[nltk_data]    |     Pavan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to C:\Users\Sai
[nltk_data]    |     Pavan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to C:\Users\Sai
[nltk_data]    |     Pavan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to C:\Users\Sai
[nltk_data]    |     Pavan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_revie

In [22]:
dataset = pd.read_csv('spam.csv',engine='python')
dataset

,Class,Text,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [23]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Class       5572 non-null   object
 1   Text        5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [24]:
dataset.isna().sum()

Class            0
Text             0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [25]:
X=dataset['Text']
Y = dataset['Class']

In [26]:
stemmer = PorterStemmer()
def process_text(value):
    no_punc=[char for char in value if char not in string.punctuation]
    new1=''.join(no_punc)
    new2=[stemmer.stem(word) for word in new1]
    new3=''.join(new2)
    return[word for word in new3.split()if word.lower()not in stopwords.words('english') ]

In [27]:
dataset["Text"].head(5).apply(process_text)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: Text, dtype: object

In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.25, random_state=87)

In [29]:
tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
stop_words = nltk.corpus.stopwords.words("english")
c_vectorizer = CountVectorizer(tokenizer=tokenizer.tokenize, stop_words=stop_words)

In [30]:
c_vectorizer.fit(iter(X_train))
X_train = c_vectorizer.transform(iter(X_train))
X_test = c_vectorizer.transform(iter(X_test))

C:\Users\Sai Pavan\AppData\Roaming\Python\Python37\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [31]:
# Encoding the Dependent Variable
labelencoder_Y = LabelEncoder()
Y= labelencoder_Y.fit_transform(Y)

In [32]:
print("Number of features before reduction : ", X_train.shape[1])
selection = VarianceThreshold(threshold=0.001)
X_train_whole = copy.deepcopy(X_train)
Y_train_whole = copy.deepcopy(Y_train)
selection.fit(X_train)
X_train = selection.transform(X_train)
X_test = selection.transform(X_test)
print("Number of features after reduction : ", X_train.shape[1])

Number of features before reduction :  7346
Number of features after reduction :  1499


In [42]:
dtc_model = DecisionTreeClassifier()
dtc_model.fit(X_train, Y_train)
dtc_pred = dtc_model.predict(X_train)

print(classification_report(Y_train, dtc_pred))

              precision    recall  f1-score   support

         ham       1.00      1.00      1.00      3624
        spam       1.00      1.00      1.00       555

    accuracy                           1.00      4179
   macro avg       1.00      1.00      1.00      4179
weighted avg       1.00      1.00      1.00      4179



In [44]:
nb_model = MultinomialNB()
nb_model.fit(X_train, Y_train)
nb_pred = nb_model .predict(X_train)
print(classification_report(Y_train, nb_pred))

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99      3624
        spam       0.97      0.96      0.96       555

    accuracy                           0.99      4179
   macro avg       0.98      0.98      0.98      4179
weighted avg       0.99      0.99      0.99      4179



In [35]:
svc_model = SVC()
svc_model.fit(X_train, Y_train)
svc_pred = svc_model.predict(X_train)
print(classification_report(Y_train, svc_pred))

              precision    recall  f1-score   support

         ham       0.99      1.00      1.00      3624
        spam       1.00      0.96      0.98       555

    accuracy                           0.99      4179
   macro avg       1.00      0.98      0.99      4179
weighted avg       0.99      0.99      0.99      4179

